In [6]:
using LinearAlgebra
using SparseArrays
using BenchmarkTools
using Random

In [7]:
A = rand(5,5)

5×5 Matrix{Float64}:
 0.642467   0.619186  0.247908  0.278075   0.876818
 0.489676   0.255611  0.307874  0.305068   0.95236
 0.91608    0.343694  0.93914   0.0618997  0.593948
 0.598057   0.476888  0.500134  0.590959   0.38941
 0.0934844  0.519326  0.145475  0.292222   0.946516

In [8]:
A[:,[1,3,5]] = round.(rand(5,3), digits=1)

5×3 Matrix{Float64}:
 0.0  0.1  0.4
 0.8  0.7  0.2
 0.9  0.3  0.1
 0.4  0.4  0.4
 0.4  1.0  0.8

In [9]:
A

5×5 Matrix{Float64}:
 0.0  0.619186  0.1  0.278075   0.4
 0.8  0.255611  0.7  0.305068   0.2
 0.9  0.343694  0.3  0.0618997  0.1
 0.4  0.476888  0.4  0.590959   0.4
 0.4  0.519326  1.0  0.292222   0.8

In [11]:
[A zeros(5,1); zeros(1,6)]

6×6 Matrix{Float64}:
 0.0  0.619186  0.1  0.278075   0.4  0.0
 0.8  0.255611  0.7  0.305068   0.2  0.0
 0.9  0.343694  0.3  0.0618997  0.1  0.0
 0.4  0.476888  0.4  0.590959   0.4  0.0
 0.4  0.519326  1.0  0.292222   0.8  0.0
 0.0  0.0       0.0  0.0        0.0  0.0

In [22]:
using JuMP
using Ipopt
using Pipe

i = 2
model = Model(Ipopt.Optimizer; add_bridges=false)
@variable(model, x[1:5])

@pipe x |> [JuMP.delete(model, x), JuMP.unregister(model, :x)]
@show JuMP.is_valid(model, x)

MethodError: MethodError: objects of type Vector{Nothing} are not callable
Use square brackets [] for indexing an Array.

In [2]:
struct elemat{T} <: AbstractVector{T}
    n::Int
    i::Int

    function elemat(n::Int, i::Int)
        @boundscheck n >= i || throw(ArgumentError("$i is larger than the size $n"))
        e = spzeros(n,1)
        e[i] = 1
        return e
    end
end

"""
    elemat(n,i,j)
"""

struct Elemat{T} <: AbstractMatrix{T}
    m :: Int
    n :: Int
    i :: Int
    j :: Int
    
    function Elemat(m::Integer, n::Integer, i::Integer, j::Integer)
        return elemat(m,i) * elemat(n,j)'
    end
end

Elemat(m::Integer, i::Integer, j::Integer) = Elemat(m, m, i, j)

Elemat

In [3]:
# Kronecker delta
function delta(i::Int, j::Int)
    return Int(i == j)
end

"""
    commat(n,i,j)
"""

struct commat{T} <: AbstractMatrix{T}
    m::Int
    n::Int

    function commat(m::Integer, n::Integer)
        v = [m * (j-1) + i for i in 1:m, j in 1:n]'[:]
        # apply this permutation to the rows (i.e. to each column) of identity matrix
        K = [[delta(i, j) for j in 1:(m*n)] for i in 1:(m*n)]
        tmp = [K[i] for i in v]
        return sparse(reduce(vcat, tmp'))
    end
end

commat(m::Integer) = commat(m, m)


"""
    nommat(n,i,j)
"""

struct nommat{T} <: AbstractMatrix{T}
    m::Int
    n::Int

    function nommat(m::Integer, n::Integer)
        mn = Int(m * n)
        return 0.5 * (sparse(1.0I, mn, mn) + commat(m, n))
    end
end

nommat(m::Integer) = nommat(m, m)

nommat

In [4]:
function vech(A::AbstractMatrix{T}) where {T}
    m = LinearAlgebra.checksquare(A)
    v = Vector{T}(undef, Int(m * (m + 1) /2))
    k = 0
    for j = 1:m, i = j:m
        @inbounds v[k+=1] = A[i, j]
    end
    return v
end

vech (generic function with 1 method)

In [5]:
struct unit_DL{T} <: AbstractVector{T}
    n::Int
    i::Int
    j::Int

    function unit_DL(n::Integer, i::Integer, j::Integer)
        @boundscheck i >= j || throw(ArgumentError("Does not satisfy i >= j"))
        u = spzeros(Int(0.5 * n * (n+1)), 1)
        u[Int( (j-1)*n + i - 0.5*j*(j-1) )] = 1.0
        return u
    end
end

In [6]:
struct elimat{T} <: AbstractMatrix{T}
    m::Int
    
    function elimat(m::Integer)
        T = tril(ones(m, m)) # Lower triangle of 1's
        f = findall(x -> x == 1, T[:]) # Get linear indexes of 1's
        k = m * (m + 1) / 2 # Row size of L
        m2 = m * m # Colunm size of L
        x = f + m2 * (0:k-1) # Linear indexes of the 1's within L'

        row = [mod(a, m2) != 0 ? mod(a, m2) : m2 for a in x]
        col = [mod(a, m2) != 0 ? div(a, m2) + 1 : div(a, m2) for a in x]
        L = sparse(row, col, ones(length(x)), m2, k)
        L = L' # Now transpose to actual L
        return L
    end
end

In [11]:
N = 3
Δx = 0.5
S = Int(N * (N + 1) / 2)
Fval = repeat([1.0, 1.0, -1.0, -1.0], outer=N - 2)

4-element Vector{Float64}:
  1.0
  1.0
 -1.0
 -1.0

In [12]:
row_i = repeat(2:(N-1), inner=4)

4-element Vector{Int64}:
 2
 2
 2
 2

In [14]:
N = 4
seq = Int.([2 + (N + 1) * (x - 1) - x * (x - 1) / 2 for x in 1:(N-1)])

3-element Vector{Int64}:
 2
 6
 9

In [ ]:
col_i = vcat(seq[1], repeat(seq[2:end-1], inner=2), seq[end])

In [ ]:
F = sparse(row_i, col_i, Fval, N, S) / 2 / Δx

In [54]:
N = 4
S = N*(N+1)/2
dx = 1/6

ii = repeat(2:(N-1), inner=4)
m = 2:N-1
mi = Int.([N*(N+1)/2 - (N-m)*(N-m+1)/2 - (N-m) for m in 2:N-1])               # this is where the xi^2 term is
mm = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) - (N-(m-2)) for m in 2:N-1])  # this is where the x_{i-1}^2 term is
mp = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) + (N-(m-1)) for m in 2:N-1])  # this is where the x_{i+1}^2 term is
jp = mi .+ 1  # this is the index of the x_{i+1}*x_i term
jm = mm .+ 1  # this is the index of the x_{i-1}*x_i term
jj = reshape([mp'; mm'; jp'; jm'],4*N-8);
vv = reshape([-ones(1,N-2); ones(1,N-2); -ones(1,N-2); ones(1,N-2)],4*N-8)/(6*dx);
F = sparse(ii,jj,vv,N,N*(N+1)/2)
# Boundary conditions
F[1,2] = -1/6/dx
F[1,N+1] = -1/6/dx
F[N,end-1] = 1/6/dx
F[N,end-2] = 1/6/dx
F

4×10 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
  ⋅   -1.0   ⋅    ⋅   -1.0    ⋅    ⋅     ⋅     ⋅     ⋅ 
 1.0   1.0   ⋅    ⋅     ⋅   -1.0   ⋅   -1.0    ⋅     ⋅ 
  ⋅     ⋅    ⋅    ⋅    1.0   1.0   ⋅     ⋅   -1.0  -1.0
  ⋅     ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    1.0   1.0    ⋅ 

In [64]:
a = range(0.8,1.2,10)

0.8:0.044444444444444446:1.2

In [67]:
zeros(1,1) .* rand(5,5)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [78]:
a  = zeros(1,1)
a[1] == 0
a = size(a)==(1,1) && a[1]==0 ? 0 : 1

0

In [86]:
typeof(NaN) == Float64

true

In [3]:
a = 1 
b = 2
c = 3 
n = 10
@info """

Constraint         = Non-Constrained
order              = $(n)
termination_status = $(a)
primal_status      = $(b)
objective_value    = $(c)
"""

┌ Info: 
│ Constraint        = Non-Constrained
│ order              = 10
│ termination_status = 1
│ primal_status      = 2
│ objective_value    = 3
└ @ Main c:\Users\tkoike3\Code\LiftAndLearn.jl\examples\scribble.ipynb:5
